# Consulta de API y Estructuración de Datos

Este notebook muestra cómo usar `requests` para consultar una API y `pandas` para estructurar y analizar los datos obtenidos.


In [4]:
# Importar las librerías necesarias
import requests
import pandas as pd
import json
from datetime import datetime

print("Librerías importadas correctamente")


Librerías importadas correctamente


## 1. Autenticación en la API

Realizamos el login para obtener el token de autenticación que usaremos en las siguientes solicitudes.


In [6]:
# URL del endpoint de login
url_login = "http://localhost:8080/api/auth/login"

# Credenciales de acceso (modificar según sea necesario)
credenciales = {
    "identification": "133553",  # Cambiar por el email/username real
    "password": "Nutabe*2020"          # Cambiar por la contraseña real
}

# Realizar la petición POST para autenticación
response = requests.post(url_login, json=credenciales)

# Verificar la respuesta
if response.status_code == 200:
    data_response = response.json()
    
    # Obtener el token (ajustar según la estructura de respuesta de la API)
    # Opciones comunes: data_response['token'], data_response['accessToken'], etc.
    token = data_response.get('token') or data_response.get('accessToken') or data_response.get('access_token')
    
    if token:
        print("Autenticación exitosa!")
        print(f"Token obtenido: {token[:50]}...")  # Mostrar solo los primeros 50 caracteres
        
        # Almacenar el token en una variable global para uso posterior
        AUTH_TOKEN = token
        
        # También almacenar el tipo de token si viene en la respuesta (Bearer, etc.)
        token_type = data_response.get('tokenType', 'Bearer') or 'Bearer'
        AUTH_TOKEN_TYPE = token_type
    else:
        print("Error: No se pudo obtener el token de la respuesta")
        print("Respuesta recibida:", json.dumps(data_response, indent=2, ensure_ascii=False))
else:
    print(f"Error en la autenticación. Status Code: {response.status_code}")
    print(f"Respuesta: {response.text}")


Autenticación exitosa!
Token obtenido: eyJhbGciOiJIUzI1NiJ9.eyJzdWIiOiIxMzM1NTMiLCJpYXQiO...


In [7]:
# Función auxiliar para obtener los headers de autenticación
def get_auth_headers():
    """
    Retorna los headers necesarios para hacer solicitudes autenticadas
    """
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'{AUTH_TOKEN_TYPE} {AUTH_TOKEN}'
    }
    return headers

# Verificar que el token esté disponible
if 'AUTH_TOKEN' in globals():
    print("Token disponible para usar en solicitudes HTTP")
    print(f"Header de autorización: {AUTH_TOKEN_TYPE} {AUTH_TOKEN[:30]}...")
else:
    print("Advertencia: El token no está disponible. Verifica la autenticación.")


Token disponible para usar en solicitudes HTTP
Header de autorización: Bearer eyJhbGciOiJIUzI1NiJ9.eyJzdWIiO...


## 2. Consulta de documentos fiscales

Realizamos una petición al endpoint de documentos fiscales usando el token de autenticación.


In [8]:
# URL del endpoint de documentos fiscales
url_fiscal_documents = "http://localhost:8080/api/fiscal-documents"

# Obtener los headers con el token de autenticación
headers = get_auth_headers()

# Realizar la petición GET al endpoint
response = requests.get(url_fiscal_documents, headers=headers)

# Verificar la respuesta
if response.status_code == 200:
    data = response.json()
    print(f"Petición exitosa! Status Code: {response.status_code}")
    print(f"Total de registros obtenidos: {len(data)}")
    
    if len(data) > 0:
        print(f"\nPrimer registro (ejemplo):")
        print(json.dumps(data[0], indent=2, ensure_ascii=False))
    
    # Almacenar los datos para uso posterior
    fiscal_documents_data = data
else:
    print(f"Error en la petición. Status Code: {response.status_code}")
    print(f"Respuesta: {response.text}")


Petición exitosa! Status Code: 200
Total de registros obtenidos: 79878

Primer registro (ejemplo):
{
  "id": 29,
  "documentType": "Documento soporte con no obligados",
  "cufeCude": "97c01c67d99caf397e2c3c1218e356772b99e8422baed5e26a616721be95f456a896e1c4c169db9791e1cda38e529882",
  "folio": "1180",
  "prefix": "DSG",
  "currency": "COP",
  "paymentForm": "2",
  "paymentMethod": "10",
  "issueDate": "2025-10-30",
  "receptionDate": null,
  "issuerNit": "900698993",
  "issuerName": "TRANSPORTES Y VOLQUETAS GONZALEZ SAS",
  "receiverNit": "3651318",
  "receiverName": "WILLIAM DE JESUS ARENAS",
  "iva": 0.0,
  "ica": 0.0,
  "ic": 0.0,
  "inc": 0.0,
  "timbre": 0.0,
  "incBags": 0.0,
  "inCarbon": 0.0,
  "inFuels": 0.0,
  "icData": 0.0,
  "icl": 0.0,
  "inpp": 0.0,
  "ibua": 0.0,
  "icui": 0.0,
  "reteIva": 0.0,
  "reteRent": 0.0,
  "reteIca": 0.0,
  "total": 1437000.0,
  "status": "Aprobado con notificación",
  "groupInfo": "Emitido"
}


## 3. Crear DataFrame con los datos

Convertimos los datos JSON obtenidos de la API en un DataFrame de pandas para facilitar su análisis y manipulación.


In [9]:
# Verificar que los datos estén disponibles
if 'fiscal_documents_data' in globals() and fiscal_documents_data:
    # Crear el DataFrame a partir de los datos JSON
    df_fiscal_documents = pd.DataFrame(fiscal_documents_data)
    
    print("DataFrame creado exitosamente!")
    print(f"Dimensiones: {df_fiscal_documents.shape[0]} filas × {df_fiscal_documents.shape[1]} columnas")
    print(f"\nColumnas disponibles: {list(df_fiscal_documents.columns)}")
    print("\n" + "="*50)
    print("\nPrimeras 5 filas del DataFrame:")
    df_fiscal_documents.head()
else:
    print("Error: No hay datos disponibles para crear el DataFrame. Verifica la consulta a la API.")


DataFrame creado exitosamente!
Dimensiones: 79878 filas × 33 columnas

Columnas disponibles: ['id', 'documentType', 'cufeCude', 'folio', 'prefix', 'currency', 'paymentForm', 'paymentMethod', 'issueDate', 'receptionDate', 'issuerNit', 'issuerName', 'receiverNit', 'receiverName', 'iva', 'ica', 'ic', 'inc', 'timbre', 'incBags', 'inCarbon', 'inFuels', 'icData', 'icl', 'inpp', 'ibua', 'icui', 'reteIva', 'reteRent', 'reteIca', 'total', 'status', 'groupInfo']


Primeras 5 filas del DataFrame:


In [ ]:
# Ver información detallada del DataFrame
print("Información del DataFrame:")
df_fiscal_documents.info()
print("\n" + "="*50)
print("\nEstadísticas descriptivas de las columnas numéricas:")
df_fiscal_documents.describe()
